In [1]:
import pandas as pd
import numpy as np
from datetime import *
from glob import glob

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.1.0


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score

In [4]:
import pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Get data

In [5]:
with open('pipeline_data/steps.dat', 'r') as handle:
    steps = handle.readline().strip().split()

In [6]:
csv_fname = glob('pipeline_data/data_????-??-??_glm.csv')[0]
df = pd.read_csv(csv_fname, dtype={'fips': str}).set_index('fips')
df.head()

case0  case1  case2  case3  case4  case5  case6  case7  case8  case9  \
fips                                                                          
01001   12.0    7.0    6.0   11.0   11.0   16.0   45.0   48.0   60.0   48.0   
01003   34.0   36.0   40.0   45.0   30.0   34.0   42.0   19.0   12.0   29.0   
01005    3.0    7.0    8.0   16.0    9.0   14.0   24.0   27.0   47.0   41.0   
01007    4.0    9.0   13.0    8.0    7.0    4.0    7.0    7.0   15.0    5.0   
01009   12.0    3.0    9.0   12.0    6.0    5.0    2.0    3.0   13.0   11.0   

       ...  death27_glm  death28_glm  death29_glm  death30_glm  death31_glm  \
fips   ...                                                                    
01001  ...         1.51         1.41         1.89         1.93         2.58   
01003  ...         2.28         2.69         3.30         3.21         3.52   
01005  ...         0.89         0.66         0.71         0.58         1.07   
01007  ...         0.76         0.61         0.79         0.84         1.30   
01009  ...         0.62         0.64         0.80         0.89         1.08   

       death32_glm  death33_glm  death34_glm  death35_glm  total_death_glm  
fips                                                                        
01001         2.54         3.32         2.94         4.43            58.31  
01003         4.07         5.29         5.24         7.84            78.91  
01005         0.68         0.85         0.66         0.95            44.75  
01007         1.06         1.32         1.13         1.64            23.09  
01009         1.07         1.22         1.16         1.64            11.22  

[5 rows x 158 columns]

## Functions

In [7]:
class tfRegr:
    
    def __init__(self, 
                 epoch=100, 
                 verbose=False, 
                 validation_split=0.2, 
                 learning_rate=.1):
        
        self.normalizer = preprocessing.Normalization()
        self.ep = epoch
        self.vb = verbose
        self.vs = validation_split
        self.lr = learning_rate
    
    def fit(self, X, y):
        self.normalizer.adapt(X)

        self.model = tf.keras.Sequential([
            self.normalizer,
            layers.Dense(units=1)
        ])

        self.model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=self.lr),
            loss='mean_absolute_error'
        )

        self.history = self.model.fit(
            X, y,
            epochs=self.ep,
            verbose=self.vb, # logging
            validation_split=self.vs # Calculate validation results on a fraction of the training data
        )
        return self.model

In [8]:
def predict(df, cur_idx):

    regrs = {
        'RF': RandomForestRegressor(min_samples_split=2), 
        'ET': ExtraTreesRegressor(min_samples_split=2), 
        'TF': tfRegr(),
    }
    
    cur = f'case{cur_idx}'

    glm = [f'case{cur_idx - 1}_glm', f'case{cur_idx}_glm'] 
    cols = [f'case{cur_idx - 1}'] + glm
    X, y = df[cols].values, df[cur].values 
    
    
    for name, regr in regrs.items():
        regr = regr.fit(X, y)

        #================ Evaluation ===============START
        y_pred = regr.predict(X).flatten()
        r2 = r2_score(y, y_pred)
        rmse = np.sqrt(((y - y_pred) ** 2).mean())
        print(f'\t{name}: r2 = {r2:.3f}, rmse = {rmse:.3f}')
        #================ Evaluation ===============START
        
        cols = [cur] + glm
        df[cur + f'_{name}_1'] = regr.predict(df[cols].values).flatten()
        
        cols = [cur + f'_{name}_1'] + glm
        df[cur + f'_{name}_2'] = regr.predict(df[cols].values).flatten()
    
    prd_cols_1 = [col for col in df.columns if col.endswith('_1')]
    prd_cols_2 = [col for col in df.columns if col.endswith('_2')]
    prd_1 = df[prd_cols_1].mean(axis=1).values
    prd_2 = df[prd_cols_2].mean(axis=1).values
    
    df.drop(prd_cols_1 + prd_cols_2, axis=1, inplace=True)
    
    return prd_1, prd_2

## Forecast Runs

In [9]:
cur_idx = len(steps) - 1
print(steps[cur_idx])

num_runs = 100
prds_1, prds_2 = [], []

for r in range(num_runs):
    print(f'run = {r}')
    prd_1, prd_2 = predict(df, cur_idx)
    prds_1.append(prd_1)
    prds_2.append(prd_2)

2020-12-05
run = 0
	RF: r2 = 0.984, rmse = 182.531
	ET: r2 = 1.000, rmse = 0.150
	TF: r2 = 0.937, rmse = 355.673
run = 1
	RF: r2 = 0.972, rmse = 238.268
	ET: r2 = 1.000, rmse = 0.088
	TF: r2 = 0.938, rmse = 355.402
run = 2
	RF: r2 = 0.957, rmse = 295.728
	ET: r2 = 1.000, rmse = 0.301
	TF: r2 = 0.937, rmse = 355.990
run = 3
	RF: r2 = 0.985, rmse = 176.624
	ET: r2 = 1.000, rmse = 0.048
	TF: r2 = 0.937, rmse = 355.794
run = 4
	RF: r2 = 0.981, rmse = 194.412
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.937, rmse = 355.917
run = 5
	RF: r2 = 0.962, rmse = 278.994
	ET: r2 = 1.000, rmse = 0.051
	TF: r2 = 0.938, rmse = 354.493
run = 6
	RF: r2 = 0.986, rmse = 169.193
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.938, rmse = 355.254
run = 7
	RF: r2 = 0.951, rmse = 316.005
	ET: r2 = 1.000, rmse = 0.112
	TF: r2 = 0.938, rmse = 355.064
run = 8
	RF: r2 = 0.947, rmse = 328.713
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.938, rmse = 355.149
run = 9
	RF: r2 = 0.967, rmse = 259.170
	ET: r2 = 1.000, rmse = 0.272
	

	TF: r2 = 0.938, rmse = 355.118
run = 80
	RF: r2 = 0.988, rmse = 154.746
	ET: r2 = 1.000, rmse = 0.043
	TF: r2 = 0.938, rmse = 354.543
run = 81
	RF: r2 = 0.978, rmse = 210.107
	ET: r2 = 1.000, rmse = 0.270
	TF: r2 = 0.937, rmse = 356.041
run = 82
	RF: r2 = 0.980, rmse = 202.086
	ET: r2 = 1.000, rmse = 0.081
	TF: r2 = 0.937, rmse = 357.725
run = 83
	RF: r2 = 0.938, rmse = 353.342
	ET: r2 = 1.000, rmse = 0.123
	TF: r2 = 0.938, rmse = 353.659
run = 84
	RF: r2 = 0.973, rmse = 232.118
	ET: r2 = 1.000, rmse = 0.150
	TF: r2 = 0.937, rmse = 356.084
run = 85
	RF: r2 = 0.970, rmse = 245.271
	ET: r2 = 1.000, rmse = 0.248
	TF: r2 = 0.937, rmse = 355.557
run = 86
	RF: r2 = 0.980, rmse = 203.194
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.937, rmse = 356.759
run = 87
	RF: r2 = 0.975, rmse = 225.657
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.937, rmse = 357.252
run = 88
	RF: r2 = 0.982, rmse = 192.395
	ET: r2 = 1.000, rmse = 0.048
	TF: r2 = 0.938, rmse = 355.085
run = 89
	RF: r2 = 0.978, rmse = 208.546


## Prepare for submission

In [10]:
# target end dates
cur_dt = datetime.strptime(steps[cur_idx], '%Y-%m-%d')
fc_dt = (cur_dt + timedelta(days=1)).date()
nxt_dt_1 = (cur_dt + timedelta(days=7)).date()
nxt_dt_2 = (cur_dt + timedelta(days=14)).date()
nxt_dts = [nxt_dt_1, nxt_dt_2]

# quantiles
qs = [.025, .1, .25, .5, .75, .9, .975]

In [11]:
df_county_prd_1 = pd.DataFrame(data=np.array(prds_1).T, index=df.index, columns=range(num_runs))
df_county_prd_2 = pd.DataFrame(data=np.array(prds_2).T, index=df.index, columns=range(num_runs))
county_dfs = [df_county_prd_1, df_county_prd_2]

### County

In [12]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    csv_fname = f'results/{steps[cur_idx]}_county_case_{w}-wk_{num_runs}.csv'
    county_df.to_csv(csv_fname, float_format='%.2f')
    
    # ========================= Point esitmate =========================START
    df_point = county_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'fips': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = county_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['fips']).sort_values('fips')\
        .rename(columns={'fips': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_county = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
print(df_forecast_county.shape)
df_forecast_county.head()

(49504, 6)


location quantile   type       value               target target_end_date
0    01001       NA  point  270.089120  1 wk ahead inc case      2020-12-12
1    01003       NA  point  797.272669  1 wk ahead inc case      2020-12-12
2    01005       NA  point   56.744817  1 wk ahead inc case      2020-12-12
3    01007       NA  point  106.539493  1 wk ahead inc case      2020-12-12
4    01009       NA  point  323.181394  1 wk ahead inc case      2020-12-12

### States

In [13]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    df_tmp = county_df.reset_index()
    df_tmp['state'] = df_tmp['fips'].apply(lambda x: x[:2])
    state_df = df_tmp.groupby(by='state').sum()
    
    # ========================= Point esitmate =========================START
    df_point = state_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'state': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = state_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['state']).sort_values('state')\
        .rename(columns={'state': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_state = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_state.head()

location quantile   type          value               target target_end_date
0       01       NA  point   21948.134942  1 wk ahead inc case      2020-12-12
1       02       NA  point    4522.556666  1 wk ahead inc case      2020-12-12
2       04       NA  point   33603.818668  1 wk ahead inc case      2020-12-12
3       05       NA  point   14953.381035  1 wk ahead inc case      2020-12-12
4       06       NA  point  145010.289438  1 wk ahead inc case      2020-12-12

### Nation

In [14]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    nation_df = pd.DataFrame(data=county_df.sum(axis=0), columns=['US']).T
    nation_df.index.name = 'nation'
    
    # ========================= Point esitmate =========================START
    df_point = nation_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'nation': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = nation_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['nation']).sort_values('nation')\
        .rename(columns={'nation': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_nation = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_nation

location quantile      type         value               target  \
0        US       NA     point  1.402659e+06  1 wk ahead inc case   
1        US    0.025  quantile  1.394396e+06  1 wk ahead inc case   
2        US      0.1  quantile  1.396292e+06  1 wk ahead inc case   
3        US     0.25  quantile  1.399950e+06  1 wk ahead inc case   
4        US      0.5  quantile  1.403024e+06  1 wk ahead inc case   
5        US     0.75  quantile  1.405464e+06  1 wk ahead inc case   
6        US      0.9  quantile  1.408275e+06  1 wk ahead inc case   
7        US    0.975  quantile  1.409322e+06  1 wk ahead inc case   
8        US       NA     point  1.477055e+06  2 wk ahead inc case   
9        US    0.025  quantile  1.461493e+06  2 wk ahead inc case   
10       US      0.1  quantile  1.467248e+06  2 wk ahead inc case   
11       US     0.25  quantile  1.471448e+06  2 wk ahead inc case   
12       US      0.5  quantile  1.477684e+06  2 wk ahead inc case   
13       US     0.75  quantile  1.483042e+06  2 wk ahead inc case   
14       US      0.9  quantile  1.487377e+06  2 wk ahead inc case   
15       US    0.975  quantile  1.490458e+06  2 wk ahead inc case   

   target_end_date  
0       2020-12-12  
1       2020-12-12  
2       2020-12-12  
3       2020-12-12  
4       2020-12-12  
5       2020-12-12  
6       2020-12-12  
7       2020-12-12  
8       2020-12-19  
9       2020-12-19  
10      2020-12-19  
11      2020-12-19  
12      2020-12-19  
13      2020-12-19  
14      2020-12-19  
15      2020-12-19

In [15]:
df_forecast = pd.concat([df_forecast_county, df_forecast_state, df_forecast_nation])
df_forecast.reset_index(drop=True, inplace=True)
df_forecast['forecast_date'] = fc_dt
cols = ['forecast_date', 'target', 'target_end_date', 'location', 'type', 'quantile', 'value']
df_forecast = df_forecast[cols]

# =============== make nagative value zero ==============
value_dezero = df_forecast['value']
value_dezero[value_dezero < 0]  = 0
df_forecast['value'] = value_dezero

In [16]:
team = 'UChicagoCHATTOPADHYAY'
model = 'UnIT'
df_forecast.to_csv(f'results/{fc_dt}-{team}-{model}_case.csv', float_format='%.2f', index=False)

In [17]:
validation = '/home/yihuang/Documents/Data/covid19-forecast-hub/code/validation/validate_single_forecast_file.py'

! python3 {validation} results/2020-11-08-UChicagoCHATTOPADHYAY-UnIT_case.csv


VALIDATING results/2020-11-08-UChicagoCHATTOPADHYAY-UnIT_case.csv
✓ results/2020-11-08-UChicagoCHATTOPADHYAY-UnIT_case.csv is valid with no errors
